In [2]:
# MobileFaceNet Architecture
import tensorflow as tf
from tensorflow.keras import layers, Model

def conv_block(x, filters, kernel, strides, name):
    x = layers.Conv2D(filters, kernel, strides=strides, padding='same', use_bias=False, name=name+'_conv')(x)
    x = layers.BatchNormalization(name=name+'_bn')(x)
    x = layers.PReLU(shared_axes=[1, 2], name=name+'_prelu')(x)
    return x

def depthwise_block(x, filters, strides, name):
    x = layers.DepthwiseConv2D(kernel_size=3, strides=strides, padding='same', use_bias=False, name=name+'_dw')(x)
    x = layers.BatchNormalization(name=name+'_dw_bn')(x)
    x = layers.PReLU(shared_axes=[1, 2], name=name+'_dw_prelu')(x)
    x = layers.Conv2D(filters, kernel_size=1, padding='same', use_bias=False, name=name+'_pw_conv')(x)
    x = layers.BatchNormalization(name=name+'_pw_bn')(x)
    x = layers.PReLU(shared_axes=[1, 2], name=name+'_pw_prelu')(x)
    return x

def MobileFaceNet(input_shape=(112, 112, 3), embedding_dim=128):
    inputs = layers.Input(shape=input_shape)
    x = conv_block(inputs, 64, 3, 2, 'conv1')
    x = depthwise_block(x, 64, 1, 'dw1')
    x = depthwise_block(x, 128, 2, 'dw2')
    x = depthwise_block(x, 128, 1, 'dw3')
    x = depthwise_block(x, 256, 2, 'dw4')
    x = depthwise_block(x, 256, 1, 'dw5')
    x = depthwise_block(x, 512, 2, 'dw6')
    for i in range(5):
        x = depthwise_block(x, 512, 1, f'dw7_{i}')
    x = layers.Conv2D(512, 1, use_bias=False, name='conv2')(x)
    x = layers.BatchNormalization(name='conv2_bn')(x)
    x = layers.PReLU(shared_axes=[1, 2], name='conv2_prelu')(x)
    x = layers.GlobalAveragePooling2D(name='GAP')(x)
    x = layers.Dense(embedding_dim, use_bias=False, name='dense')(x)
    x = layers.BatchNormalization(name='dense_bn')(x)
    return Model(inputs, x, name='MobileFaceNet')

model = MobileFaceNet()
model.summary()


Model: "MobileFaceNet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 112, 112, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_conv (Conv2D)             │ (None, 56, 56, 64)     │         1,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 56, 56, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_prelu (PReLU)             │ (None, 56, 56, 64)     │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw1_dw (DepthwiseConv2D)        │ (None, 56, 56, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw1_dw_bn (BatchNormalization)  │ (None, 56, 56, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw1_dw_prelu (PReLU)            │ (None, 56, 56, 64)     │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw1_pw_conv (Conv2D)            │ (None, 56, 56, 64)     │         4,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw1_pw_bn (BatchNormalization)  │ (None, 56, 56, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw1_pw_prelu (PReLU)            │ (None, 56, 56, 64)     │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw2_dw (DepthwiseConv2D)        │ (None, 28, 28, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw2_dw_bn (BatchNormalization)  │ (None, 28, 28, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw2_dw_prelu (PReLU)            │ (None, 28, 28, 64)     │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw2_pw_conv (Conv2D)            │ (None, 28, 28, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw2_pw_bn (BatchNormalization)  │ (None, 28, 28, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw2_pw_prelu (PReLU)            │ (None, 28, 28, 128)    │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw3_dw (DepthwiseConv2D)        │ (None, 28, 28, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw3_dw_bn (BatchNormalization)  │ (None, 28, 28, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw3_dw_prelu (PReLU)            │ (None, 28, 28, 128)    │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw3_pw_conv (Conv2D)            │ (None, 28, 28, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw3_pw_bn (BatchNormalization)  │ (None, 28, 28, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw3_pw_prelu (PReLU)            │ (None, 28, 28, 128)    │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw4_dw (DepthwiseConv2D)        │ (None, 14, 14, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw4_dw_bn (BatchNormalization)  │ (None, 14, 14, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dw4_dw_prelu (PReLU)            │ (None, 14, 14, 128)    │           12

 Total params: 1,969,472 (7.51 MB)

 Trainable params: 1,953,344 (7.45 MB)

 Non-trainable params: 16,128 (63.00 KB)

In [3]:
# Training Phase (Feature + KNN)
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import pickle

train_dir = "data/train"
X_train, y_train = [], []

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return None
    img = cv2.resize(img, (112, 112))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    return np.expand_dims(img, axis=0)

print("Loading training images...")
for person_name in os.listdir(train_dir):
    person_folder = os.path.join(train_dir, person_name)
    if not os.path.isdir(person_folder):
        continue

    for img_name in os.listdir(person_folder):
        img_path = os.path.join(person_folder, img_name)
        img = preprocess_image(img_path)
        if img is None:
            continue
        embedding = model.predict(img)[0]
        X_train.append(embedding)
        y_train.append(person_name)

X_train = np.array(X_train)
y_train = np.array(y_train)
print("Total embeddings:", X_train.shape)

encoder = LabelEncoder()
y_enc = encoder.fit_transform(y_train)

knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn.fit(X_train, y_enc)

# Save Models
model.save("mobilefacenet_feature_extractor.h5")
pickle.dump(knn, open("knn_model.pkl", "wb"))
pickle.dump(encoder, open("label_encoder.pkl", "wb"))

print("Training complete & models saved!")


Loading training images...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


Total embeddings: (15, 128)
Training complete & models saved!


In [4]:
# Testing Phase
import numpy as np
import cv2
from tensorflow.keras.models import load_model
import pickle

model = load_model("mobilefacenet_feature_extractor.h5")
knn = pickle.load(open("knn_model.pkl", "rb"))
encoder = pickle.load(open("label_encoder.pkl", "rb"))

test_dir = "data/test"

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (112, 112))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    return np.expand_dims(img, axis=0)

for img_name in os.listdir(test_dir):
    img_path = os.path.join(test_dir, img_name)
    img = preprocess_image(img_path)
    embedding = model.predict(img)[0]
    pred = knn.predict([embedding])[0]
    name = encoder.inverse_transform([pred])[0]
    print(f"{img_name} → Predicted: {name}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step
test_person4.jpeg → Predicted: person4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
p.jpg → Predicted: person3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
test_person3.png → Predicted: person4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
test_person2.jpeg → Predicted: person4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
test_person1.jpeg → Predicted: person4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
m.webp → Predicted: person2


In [5]:
# Real-time Recognition
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import pickle

model = load_model("mobilefacenet_feature_extractor.h5")
knn = pickle.load(open("knn_model.pkl", "rb"))
encoder = pickle.load(open("label_encoder.pkl", "rb"))

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    face = cv2.resize(frame, (112, 112))
    img = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    embedding = model.predict(img)[0]
    pred = knn.predict([embedding])[0]
    name = encoder.inverse_transform([pred])[0]

    cv2.putText(frame, name, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.imshow("MobileFaceNet + KNN", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 381ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━